# DEMO: Serenity Risk Measures API

In this demo notebook we describe one possible use case of the risk-measure interface: approximating the optimal hedge quantity for a given portfolio.

We assume the portfolio to be composed only by Bitcoin and Ether and we use a short dated put option for our hedge. We will use a bitcoin option to hedge both positions 

In [ ]:
%%capture --no-stderr --no-display
%load_ext autoreload
%autoreload 2

In [ ]:

# Standard imports
from datetime import datetime
from os import getenv
from uuid import UUID
from uuid import uuid4

# External packages
import pandas as pd
import matplotlib.pyplot as plt

# Serenity imports
from serenity_sdk.widgets import ConnectWidget
from serenity_sdk.renderers.derivatives.widget_tools import OptionChooser
from serenity_sdk.types.common import Portfolio

from serenity_types.portfolio.core import AssetPosition
from serenity_types.pricing.derivatives.options.valuation import OptionValuation, OptionValuationRequest
from serenity_types.refdata.options import OptionType
from serenity_types.risk import measures as risk_measures
from serenity_sdk.types.measures import RiskMeasureContext

In [ ]:
# if you want to auto-connect, set this environment variable to your desired default
connect_widget = ConnectWidget(getenv('SERENITY_CONFIG_ID', None))

### Parameters

In [ ]:
# Portfolio parameters

from math import e


bitcoin_name = "Bitcoin"
bitcoin_id = "78e2e8e2-419d-4515-9b6a-3d5ff1448e89" # This is Bitcoin
bitcoin_quantity = 1.0
bitcoin_price = 25000.0

ether_name = "Ether"
ether_id = "95b89dfd-c5f6-41d6-83c6-a9d97e0ac361" # This is Ether
ether_quantity = bitcoin_quantity * 4.0
ether_price = 1700.0

option_name = "Put Option"
option_initial_quantity = 0.2

# We want to hedge against this specific risk measure
target_measure_tag="CVaR 95%"

# Chart and format parameters
figsize=(9, 4)
currency_format = '{:,.2f} $'.format
percent_format = '{:,.2%}'.format
pct_formatter = plt.FuncFormatter(lambda x, loc: percent_format(x))

In [ ]:
# We use the current time
as_of_datetime = datetime.utcnow()

as_of_datetime

In [ ]:
# Risk measure parameters

measures = [
    risk_measures.MeasureParameters(
        tag="CVaR 99%",
        measure_type=risk_measures.MeasureType.CVAR,
        confidence_level="0.99",
    ),
    risk_measures.MeasureParameters(
        tag="VaR 99%",
        measure_type=risk_measures.MeasureType.VAR,
        confidence_level="0.99",
    ),
    risk_measures.MeasureParameters(
        tag=target_measure_tag,
        measure_type=risk_measures.MeasureType.CVAR,
        confidence_level="0.95",
    ),
    risk_measures.MeasureParameters(
        tag="Downside Dev",
        measure_type=risk_measures.MeasureType.DOWNDEV,
    ),
    risk_measures.MeasureParameters(
        tag="Standard Dev",
        measure_type=risk_measures.MeasureType.STDEV,
    ),
    risk_measures.MeasureParameters(
        tag="Upside Dev",
        measure_type=risk_measures.MeasureType.UPDEV,
    ),
    risk_measures.MeasureParameters(
        tag="CGaR 95%",
        measure_type=risk_measures.MeasureType.CGAR,
        confidence_level="0.95",
    ),
    risk_measures.MeasureParameters(
        tag="GaR 99%",
        measure_type=risk_measures.MeasureType.GAR,
        confidence_level="0.99",
    ),
    risk_measures.MeasureParameters(
        tag="CGaR 99%",
        measure_type=risk_measures.MeasureType.CGAR,
        confidence_level="0.99",
    ),
]


risk_computation_request = risk_measures.RiskComputationRequest(
    lookback_days=365, sampling_hours=1, horizon_scale="24.0", measures=measures
)

# We split risk measures into two groups: dispersion and concentration
dispersion_tags = ["Standard Dev", "Downside Dev", "Upside Dev"]
concetration_tags = ["VaR 99%", "CVaR 99%", "GaR 99%", "CGaR 99%"]

In [ ]:

api = connect_widget.api

## Choose the relevant put option

In [ ]:
# First we select all available options
option_chooser = OptionChooser(api)

listed_options = option_chooser.data

listed_options.head(3)

In [ ]:
# We select only Bitcoin put options
bitcoin_put_options = listed_options[
    (listed_options["underlier_asset_id"] == UUID(bitcoin_id))
    & (option_chooser.data["option_type"] == OptionType.PUT)
]

bitcoin_put_options.head(3)

In [ ]:
# Then we choose the mid-maturity options

expiry_datetimes = sorted(bitcoin_put_options["expiry_datetime"].unique())

# Let's make sure that we have enough option expiries
assert len(expiry_datetimes) >= 3

# We choose the first third of the expiries
choosen_expiry = expiry_datetimes[int(len(expiry_datetimes)/3)]

choosen_expiry

In [ ]:
# Then we look for the mid strike 
term_options = bitcoin_put_options[bitcoin_put_options["expiry_datetime"] == choosen_expiry]

strike_prices = sorted(term_options["strike_price"].unique())

# Let's make sure that we have enough strike prices
assert len(strike_prices) >= 3

choosen_strike = strike_prices[int(len(strike_prices)/2)]

choosen_strike

In [ ]:
# Finally we select the option, as the first option with the choosen strike price and expiry

selected_option = term_options[term_options["strike_price"] == choosen_strike].iloc[0]

selected_option

In [ ]:
option_id = str(selected_option["asset_id"])

option_id

## Compute risk and components for the initial portfolio

### Create the hedged portfolio

In [ ]:
# The default option valuation object


the_default_optval = OptionValuation(
    valuation_id=str(uuid4()), qty=1, option_asset_id=UUID(option_id)
)
opt_val_res = api.pricer().compute_option_valuations(
    request=OptionValuationRequest(
        as_of_time=as_of_datetime, discounting_method="CURVE",
        options=[the_default_optval])
)

opt_val_res

In [ ]:
option_pv = opt_val_res[0].pv

option_pv

In [ ]:
portfolio_1_value = bitcoin_quantity * bitcoin_price + ether_quantity * ether_price + option_initial_quantity * option_pv

portfolio_1_value

In [ ]:
portfolio_1 = Portfolio(
    assets={
        UUID(bitcoin_id): bitcoin_quantity,         # Bitcoin
        UUID(ether_id): ether_quantity,             # Ether
        UUID(option_id): option_initial_quantity,    # Put option
    }
)

In [ ]:
ctx = RiskMeasureContext(
    request=risk_computation_request,
    as_of_time=as_of_datetime,
)
risk_response_1 = api.risk().compute_risk_measures(ctx, portfolio_1)

### Show risk, the P&L distribution and the risk components

In [ ]:
# Cash risk measure values

pd.Series(risk_response_1.portfolio.values).to_frame().applymap(currency_format)

In [ ]:
portfolio_measures_1 = pd.Series(risk_response_1.portfolio.values)/portfolio_1_value
portfolio_measures_1.name = "Initial Portfolio"


portfolio_measures_1.to_frame().applymap(percent_format)

In [ ]:
ax = portfolio_measures_1.plot.bar(figsize=figsize, title="Initial portfolio risk measures")

ax.yaxis.set_major_formatter(pct_formatter)

#### Plot the P&L distribution

In [ ]:
plt.figure(figsize=figsize)
measure_colors = ["r", "g", "y", "k"]
plt.hist(risk_response_1.pnl_risk_scenarios, bins=200)
for idx, (measure_name, measure_value) in enumerate(
    pd.Series(risk_response_1.portfolio.values)[concetration_tags].items()
):
    sign_multiplier = 1 if "GaR" in measure_name else -1
    plt.axvline(
        sign_multiplier * measure_value,
        color=measure_colors[idx],
        linestyle="dashed",
        linewidth=1,
        label=measure_name,
    )
plt.xlabel("PnL")
plt.title("Initial portfolio P&L distribution")

plt.legend()
plt.show()

### Look at risk components

While the components from Bitcoin and Ether are positive, the put option component is negative. 
This confirms that the put option can be used as portfolio hedge 


In [ ]:
risk_components_1 =pd.DataFrame({
    str(component.asset_id): pd.Series(component.values) for component in risk_response_1.contributions
}).rename(columns={bitcoin_id: bitcoin_name, ether_id: ether_name, option_id: option_name})

risk_components_1.applymap(currency_format)


## Create a second portfolio with an improved hedge with respect to the chosen measure

### Compute the hedge ratios based on the risk componets

In [ ]:
# We compute the sum of the Bitcoin and Ether components

long_portfolio_component = risk_components_1[[bitcoin_name, ether_name]].sum(axis="columns")
long_portfolio_component.name = "Long portfolio risk contribution"

long_portfolio_component.to_frame().applymap(currency_format)

In [ ]:
hedge_ratios = -  long_portfolio_component.div(risk_components_1[option_name], axis="index")
hedge_ratios.name = "Hedge ratios"

hedge_ratios.to_frame().applymap(lambda x: "{:.2f}".format(x))

In [ ]:
hedge_ratios.plot.bar(figsize=figsize, title="Hedge ratios")

## Create a second portfolio with improved hedge

In [ ]:
revised_option_quantity = hedge_ratios[target_measure_tag] * option_initial_quantity

revised_option_quantity

In [ ]:
portfolio_2 = Portfolio(
    assets={
        UUID(bitcoin_id): bitcoin_quantity,         # Bitcoin
        UUID(ether_id): ether_quantity,             # Ether
        UUID(option_id): revised_option_quantity,    # Put option
    }
)

In [ ]:
portfolio_2_value = bitcoin_quantity * bitcoin_price + ether_quantity * ether_price + revised_option_quantity * option_pv

portfolio_2_value

#### Call the risk API for the revised portfolio

In [ ]:
risk_response_2 = api.risk().compute_risk_measures(ctx, portfolio_2)

In [ ]:
portfolio_measures_2 = pd.Series(risk_response_2.portfolio.values)/portfolio_1_value
portfolio_measures_2.name = "Revised Portfolio"


portfolio_measures_2.to_frame().applymap(percent_format)

#### We notice we now have a striking asymmetry between risk and upside

In [ ]:
ax = portfolio_measures_2.plot.bar(figsize=figsize, title="Revised portfolio risk measures")

ax.yaxis.set_major_formatter(pct_formatter)

### Looking at the revised distribution

In [ ]:
plt.figure(figsize=figsize)
measure_colors = ["r", "g", "y", "k"]
plt.hist(risk_response_2.pnl_risk_scenarios, bins=200)
for idx, (measure_name, measure_value) in enumerate(
    pd.Series(risk_response_2.portfolio.values)[concetration_tags].items()
):
    sign_multiplier = 1 if "GaR" in measure_name else -1
    plt.axvline(
        sign_multiplier * measure_value,
        color=measure_colors[idx],
        linestyle="dashed",
        linewidth=1,
        label=measure_name,
    )
plt.xlabel("PnL")
plt.title("Revised portfolio P&L distribution")

plt.legend()
plt.show()